In [ ]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

In [ ]:
import pandas as pd
import numpy as np
from skforecast.datasets import fetch_dataset
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries
from skforecast.model_selection_multiseries import bayesian_search_forecaster_multiseries
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from skforecast.metrics import mean_absolute_scaled_error

In [ ]:
import pandas as pd
import numpy as np
from skforecast.datasets import fetch_dataset
from skforecast.model_selection_multiseries.model_selection_multiseries import _calculate_metrics_multiseries
from skforecast.metrics import add_y_train_argument
from sklearn.metrics import mean_absolute_error
from skforecast.metrics import mean_absolute_scaled_error

In [ ]:
data = pd.DataFrame(
    data={
        "item_1": [
            8.253175, 22.777826, 27.549099, 25.895533, 21.379238, 21.106643,
            20.533871, 20.069327, 20.006161, 21.620184, 21.717691, 21.751748,
            21.758617, 20.784194, 18.976196, 20.228468, 26.636444, 29.245869,
            24.772249, 24.018768, 22.503533, 20.794986, 23.981037, 28.018830,
            28.747482, 23.908368, 21.423930, 24.786455, 24.615778, 27.388275,
            25.724191, 22.825491, 23.066582, 23.788066, 23.360304, 23.119966,
            21.763739, 23.008517, 22.861086, 22.807790, 23.424717, 22.208947,
            19.558775, 20.788390, 23.619240, 25.061150, 27.646380, 25.609772,
            22.504042, 20.838095
        ],
        "item_2": [
            21.047727, 26.578125, 31.751042, 24.567708, 18.191667, 17.812500,
            19.510417, 24.098958, 20.223958, 19.161458, 16.042708, 14.815625,
            17.031250, 17.009375, 17.096875, 19.255208, 28.060417, 28.779167,
            19.265625, 19.178125, 19.688542, 21.690625, 25.332292, 26.675000,
            26.611458, 19.759375, 20.038542, 24.680208, 25.032292, 28.111458,
            21.542708, 16.605208, 18.593750, 20.667708, 21.977083, 29.040625,
            18.979167, 18.459375, 17.295833, 17.282292, 20.844792, 19.858333,
            18.446875, 19.239583, 19.903125, 22.970833, 28.195833, 20.221875,
            19.176042, 21.991667
        ],
        "item_3": [
            19.429739, 28.009863, 32.078922, 27.252276, 20.357737, 19.879148,
            18.043499, 26.287368, 16.315997, 21.772584, 18.729748, 12.552534,
            18.996209, 18.534327, 15.418361, 16.304852, 30.076258, 28.886334,
            20.286651, 21.367727, 20.248170, 19.799975, 25.931558, 27.698196,
            30.725005, 19.573577, 23.310162, 24.959233, 24.399246, 29.094136,
            22.639513, 18.372362, 21.256450, 22.430527, 19.575067, 31.767626,
            20.086271, 21.380186, 17.553807, 17.369879, 21.829746, 16.208510,
            25.067215, 21.863615, 17.887458, 23.005424, 25.013939, 22.142083,
            23.673005, 25.238480
        ],
    },
    index=pd.date_range(start="2012-01-01", end="2012-02-19"),
)


predictions_different_lenght = pd.DataFrame(
    data={
        "item_1": [
            25.849411, 24.507137, 23.885447, 23.597504, 23.464140, 23.402371,
            23.373762, 23.360511, 23.354374, 23.351532, 23.354278, 23.351487,
            23.350195, 23.349596, 23.349319, 23.349190, 23.349131, 23.349103,
            23.349090, 23.349084, 23.474207, 23.407034, 23.375922, 23.361512,
            23.354837
        ],
        "item_2": [
            24.561460, 23.611980, 23.172218, 22.968536, 22.874199, 22.830506,
            22.810269, 22.800896, 22.796555, 22.794544, 22.414996, 22.617821,
            22.711761, 22.755271, 22.775423, 22.784756, 22.789079, 22.791082,
            22.792009, 22.792439, 21.454419, 22.172918, 22.505700, 22.659831,
            22.731219
        ],
        "item_3": [
            26.168069, 24.057472, 23.079925, 22.627163, 22.417461, 22.320335,
            22.275350, 22.254515, 22.244865, 22.240395, 21.003848, 21.665604,
            np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,
            np.nan, np.nan, np.nan, np.nan, np.nan
        ],
    },
    index=pd.date_range(start="2012-01-26", periods=25)
)


span_index = span_index = pd.date_range(start="2012-01-01", end="2012-02-19", freq="D")

folds = [
    [[0, 25], [24, 25], [25, 35], [25, 35], False],
    [[0, 25], [34, 35], [35, 45], [35, 45], False],
    [[0, 25], [44, 45], [45, 50], [45, 50], False],
]

levels = ["item_1", "item_2", "item_3"]

In [ ]:
def test_calculate_metrics_multiseries_output_when_aggregated_metric_and_predictions_have_different_length(
    metrics=[mean_absolute_error, mean_absolute_scaled_error]
):

    metrics = [add_y_train_argument(metric) for metric in metrics]
    results = _calculate_metrics_multiseries(
        series=data,
        predictions=predictions_different_lenght,
        folds=folds,
        span_index=span_index,
        metrics=metrics,
        levels=levels,
        add_aggregated_metric=True,
    )

    expected = pd.DataFrame(
        data={
            "levels": [
                "item_1",
                "item_2",
                "item_3",
                "average",
                "weighted_average",
                "pooling",
            ],
            "mean_absolute_error": [
                1.477567,
                3.480129,
                3.173683,
                2.710460,
                2.613332,
                2.613332,
            ],
            "mean_absolute_scaled_error": [
                0.610914,
                1.170113,
                0.707757,
                0.829595,
                0.855141,
                0.793768,
            ],
        }
    )
    display(results)
    pd.testing.assert_frame_equal(results, expected)


test_calculate_metrics_multiseries_output_when_aggregated_metric_and_predictions_have_different_length()

In [ ]:
(data.iloc[25:] - predictions).abs().stack().mean()

In [ ]:
np.average([1.477567, 3.480129, 3.173683], weights=[25, 25, 12])

In [ ]:
predictions.notna().sum()

In [186]:
# Unit test backtesting_forecaster_multiseries
# ==============================================================================
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error


from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.ForecasterAutoregMultiSeriesCustom import ForecasterAutoregMultiSeriesCustom
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import backtesting_forecaster_multivariate
from skforecast.model_selection_multiseries.model_selection_multiseries import _bayesian_search_optuna_multiseries


# Fixtures
from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import series
THIS_DIR = Path("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests")
series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}
series_with_nans = series.copy()
series_with_nans.iloc[:10, series_with_nans.columns.get_loc('l2')] = np.nan

In [162]:
def test_output_evaluate_grid_hyperparameters_multiseries_ForecasterAutoregMultiSeries_multiple_metrics_aggregated_with_mocked():
    """
    Test output of _evaluate_grid_hyperparameters_multiseries in ForecasterAutoregMultiSeries 
    with mocked when multiple metrics (mocked done in Skforecast v0.6.0).
    """
    forecaster = ForecasterAutoregMultiSeries(
                     regressor          = Ridge(random_state=123),
                     lags               = 2, 
                     encoding           = 'onehot',
                     transformer_series = None
                 )

    steps = 3
    n_validation = 12
    lags_grid = [2, 4]
    param_grid = [{'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}]

    results = _evaluate_grid_hyperparameters_multiseries(
                  forecaster         = forecaster,
                  series             = series,
                  param_grid         = param_grid,
                  steps              = steps,
                  metric             = ['mean_absolute_error', 'mean_absolute_scaled_error'],
                  aggregate_metric   = ['weighted_average', 'average', 'pooling'],
                  initial_train_size = len(series) - n_validation,
                  fixed_train_size   = False,
                  levels             = None,
                  exog               = None,
                  lags_grid          = lags_grid,
                  refit              = False,
                  return_best        = False,
                  verbose            = False,
                  show_progress     = False,
              )
    
    expected_results = pd.DataFrame({
        "levels": {
            0: ["l1", "l2"],
            1: ["l1", "l2"],
            2: ["l1", "l2"],
            3: ["l1", "l2"],
            4: ["l1", "l2"],
            5: ["l1", "l2"],
        },
        "lags": {
            0: np.array([1, 2, 3, 4]),
            1: np.array([1, 2, 3, 4]),
            2: np.array([1, 2, 3, 4]),
            3: np.array([1, 2]),
            4: np.array([1, 2]),
            5: np.array([1, 2]),
        },
        "lags_label": {
            0: np.array([1, 2, 3, 4]),
            1: np.array([1, 2, 3, 4]),
            2: np.array([1, 2, 3, 4]),
            3: np.array([1, 2]),
            4: np.array([1, 2]),
            5: np.array([1, 2]),
        },
        "params": {
            0: {"alpha": 0.01},
            1: {"alpha": 0.1},
            2: {"alpha": 1},
            3: {"alpha": 1},
            4: {"alpha": 0.1},
            5: {"alpha": 0.01},
        },
        "mean_absolute_error__weighted_average": {
            0: 0.20968100547390048,
            1: 0.20969259864077977,
            2: 0.20977945397058564,
            3: 0.21077344921320568,
            4: 0.21078653208835063,
            5: 0.21078779920557153,
        },
        "mean_absolute_error__average": {
            0: 0.20968100547390048,
            1: 0.20969259864077974,
            2: 0.20977945397058564,
            3: 0.21077344921320565,
            4: 0.21078653208835063,
            5: 0.21078779920557153,
        },
        "mean_absolute_error__pooling": {
            0: 0.20968100547390045,
            1: 0.2096925986407798,
            2: 0.20977945397058564,
            3: 0.21077344921320565,
            4: 0.21078653208835063,
            5: 0.21078779920557153,
        },
        "mean_absolute_scaled_error__weighted_average": {
            0: 0.7969369551529275,
            1: 0.7969838748911608,
            2: 0.7973389652448446,
            3: 0.8009631048212882,
            4: 0.8009302953795885,
            5: 0.8009249124659391,
        },
        "mean_absolute_scaled_error__average": {
            0: 0.7969369551529275,
            1: 0.7969838748911608,
            2: 0.7973389652448445,
            3: 0.8009631048212883,
            4: 0.8009302953795885,
            5: 0.8009249124659391,
        },
        "mean_absolute_scaled_error__pooling": {
            0: 0.7809734688246502,
            1: 0.7810166484905049,
            2: 0.7813401480275807,
            3: 0.7850423618302551,
            4: 0.785091090032226,
            5: 0.7850958095104122,
        },
        "alpha": {0: 0.01, 1: 0.1, 2: 1.0, 3: 1.0, 4: 0.1, 5: 0.01},
    })

    pd.testing.assert_frame_equal(results, expected_results)

test_output_evaluate_grid_hyperparameters_multiseries_ForecasterAutoregMultiSeries_multiple_metrics_aggregated_with_mocked()

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:383: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


6 models compared for 2 level(s). Number of iterations: 6.


In [ ]:
forecaster = ForecasterAutoregMultiSeries(
                    regressor          = Ridge(random_state=123),
                    lags               = 2, 
                    encoding           = 'onehot',
                    transformer_series = None
                )

steps = 3
n_validation = 12
lags_grid = [2, 4]
param_grid = [{'alpha': 0.01}, {'alpha': 0.1}, {'alpha': 1}]

results = _evaluate_grid_hyperparameters_multiseries(
                forecaster         = forecaster,
                series             = series,
                param_grid         = param_grid,
                steps              = steps,
                metric             = ['mean_absolute_error', 'mean_absolute_scaled_error'],
                aggregate_metric   = ['weighted_average', 'average', 'pooling'],
                initial_train_size = len(series) - n_validation,
                fixed_train_size   = False,
                levels             = None,
                exog               = None,
                lags_grid          = lags_grid,
                refit              = False,
                return_best        = False,
                verbose            = False,
                show_progress     = False
            )

results.to_dict()

In [160]:
pd.DataFrame({
        "levels": {
            0: ["l1", "l2"],
            1: ["l1", "l2"],
            2: ["l1", "l2"],
            3: ["l1", "l2"],
            4: ["l1", "l2"],
            5: ["l1", "l2"],
        },
        "lags": {
            0: np.array([1, 2, 3, 4]),
            1: np.array([1, 2, 3, 4]),
            2: np.array([1, 2, 3, 4]),
            3: np.array([1, 2]),
            4: np.array([1, 2]),
            5: np.array([1, 2]),
        },
        "lags_label": {
            0: np.array([1, 2, 3, 4]),
            1: np.array([1, 2, 3, 4]),
            2: np.array([1, 2, 3, 4]),
            3: np.array([1, 2]),
            4: np.array([1, 2]),
            5: np.array([1, 2]),
        },
        "params": {
            0: {"alpha": 0.01},
            1: {"alpha": 0.1},
            2: {"alpha": 1},
            3: {"alpha": 1},
            4: {"alpha": 0.1},
            5: {"alpha": 0.01},
        },
        "mean_absolute_error__weighted_average": {
            0: 0.20968100547390048,
            1: 0.20969259864077977,
            2: 0.20977945397058564,
            3: 0.21077344921320568,
            4: 0.21078653208835063,
            5: 0.21078779920557153,
        },
        "mean_absolute_error__average": {
            0: 0.20968100547390048,
            1: 0.20969259864077974,
            2: 0.20977945397058564,
            3: 0.21077344921320565,
            4: 0.21078653208835063,
            5: 0.21078779920557153,
        },
        "mean_absolute_error__pooling": {
            0: 0.20968100547390045,
            1: 0.2096925986407798,
            2: 0.20977945397058564,
            3: 0.21077344921320565,
            4: 0.21078653208835063,
            5: 0.21078779920557153,
        },
        "mean_absolute_scaled_error__weighted_average": {
            0: 0.7969369551529275,
            1: 0.7969838748911608,
            2: 0.7973389652448446,
            3: 0.8009631048212882,
            4: 0.8009302953795885,
            5: 0.8009249124659391,
        },
        "mean_absolute_scaled_error__average": {
            0: 0.7969369551529275,
            1: 0.7969838748911608,
            2: 0.7973389652448445,
            3: 0.8009631048212883,
            4: 0.8009302953795885,
            5: 0.8009249124659391,
        },
        "mean_absolute_scaled_error__pooling": {
            0: 0.7809734688246502,
            1: 0.7810166484905049,
            2: 0.7813401480275807,
            3: 0.7850423618302551,
            4: 0.785091090032226,
            5: 0.7850958095104122,
        },
        "alpha": {0: 0.01, 1: 0.1, 2: 1.0, 3: 1.0, 4: 0.1, 5: 0.01},
    }).equals(results)

True

In [179]:
def test_output_grid_search_forecaster_multiseries_ForecasterAutoregMultiSeries_with_mocked():
    """
    Test output of grid_search_forecaster_multiseries in ForecasterAutoregMultiSeries 
    with mocked (mocked done in Skforecast v0.5.0)
    """
    forecaster = ForecasterAutoregMultiSeries(
                     regressor          = Ridge(random_state=123),
                     lags               = 2, 
                     encoding           = 'onehot',
                     transformer_series = None
                 )

    steps = 3
    n_validation = 12
    lags_grid = [2, 4]
    param_grid = {'alpha': [0.01, 0.1, 1]}

    results = grid_search_forecaster_multiseries(
                  forecaster          = forecaster,
                  series              = series,
                  param_grid          = param_grid,
                  steps               = steps,
                  metric              = ['mean_absolute_error', 'mean_absolute_scaled_error'],
                  aggregate_metric   = ['weighted_average', 'average', 'pooling'],
                  initial_train_size  = len(series) - n_validation,
                  fixed_train_size    = False,
                  levels              = None,
                  exog                = None,
                  lags_grid           = lags_grid,
                  refit               = False,
                  return_best         = False,
                  verbose             = False,
                  show_progress       = False,
              )
    
    expected_results = pd.DataFrame({
        'levels': {0: ['l1', 'l2'],
        1: ['l1', 'l2'],
        2: ['l1', 'l2'],
        3: ['l1', 'l2'],
        4: ['l1', 'l2'],
        5: ['l1', 'l2']},
        'lags': {0: np.array([1, 2, 3, 4]),
        1: np.array([1, 2, 3, 4]),
        2: np.array([1, 2, 3, 4]),
        3: np.array([1, 2]),
        4: np.array([1, 2]),
        5: np.array([1, 2])},
        'lags_label': {0: np.array([1, 2, 3, 4]),
        1: np.array([1, 2, 3, 4]),
        2: np.array([1, 2, 3, 4]),
        3: np.array([1, 2]),
        4: np.array([1, 2]),
        5: np.array([1, 2])},
        'params': {0: {'alpha': 0.01},
        1: {'alpha': 0.1},
        2: {'alpha': 1},
        3: {'alpha': 1},
        4: {'alpha': 0.1},
        5: {'alpha': 0.01}},
        'mean_absolute_error__weighted_average': {0: 0.20968100547390048,
        1: 0.20969259864077977,
        2: 0.20977945397058564,
        3: 0.21077344921320568,
        4: 0.21078653208835063,
        5: 0.21078779920557153},
        'mean_absolute_error__average': {0: 0.20968100547390048,
        1: 0.20969259864077974,
        2: 0.20977945397058564,
        3: 0.21077344921320565,
        4: 0.21078653208835063,
        5: 0.21078779920557153},
        'mean_absolute_error__pooling': {0: 0.20968100547390045,
        1: 0.2096925986407798,
        2: 0.20977945397058564,
        3: 0.21077344921320565,
        4: 0.21078653208835063,
        5: 0.21078779920557153},
        'mean_absolute_scaled_error__weighted_average': {0: 0.7969369551529275,
        1: 0.7969838748911608,
        2: 0.7973389652448446,
        3: 0.8009631048212882,
        4: 0.8009302953795885,
        5: 0.8009249124659391},
        'mean_absolute_scaled_error__average': {0: 0.7969369551529275,
        1: 0.7969838748911608,
        2: 0.7973389652448445,
        3: 0.8009631048212883,
        4: 0.8009302953795885,
        5: 0.8009249124659391},
        'mean_absolute_scaled_error__pooling': {0: 0.7809734688246502,
        1: 0.7810166484905049,
        2: 0.7813401480275807,
        3: 0.7850423618302551,
        4: 0.785091090032226,
        5: 0.7850958095104122},
        'alpha': {0: 0.01, 1: 0.1, 2: 1.0, 3: 1.0, 4: 0.1, 5: 0.01}
    })

    pd.testing.assert_frame_equal(results, expected_results)

test_output_grid_search_forecaster_multiseries_ForecasterAutoregMultiSeries_with_mocked()

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:383: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


6 models compared for 2 level(s). Number of iterations: 6.


In [176]:
forecaster = ForecasterAutoregMultiSeries(
                    regressor          = Ridge(random_state=123),
                    lags               = 2, 
                    encoding           = 'onehot',
                    transformer_series = None
                )

steps = 3
n_validation = 12
lags_grid = [2, 4]
param_grid = {'alpha': [0.01, 0.1, 1]}

results = grid_search_forecaster_multiseries(
                forecaster          = forecaster,
                series              = series,
                param_grid          = param_grid,
                steps               = steps,
                metric              = ['mean_absolute_error', 'mean_absolute_scaled_error'],
                aggregate_metric    = ['weighted_average', 'average', 'pooling'],
                initial_train_size  = len(series) - n_validation,
                fixed_train_size    = False,
                levels              = None,
                exog                = None,
                lags_grid           = lags_grid,
                refit               = False,
                return_best         = False,
                verbose             = False,
                show_progress       = False,
            )

results.to_dict()

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:383: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


6 models compared for 2 level(s). Number of iterations: 6.


,levels,lags,lags_label,params,mean_absolute_error__weighted_average,mean_absolute_error__average,mean_absolute_error__pooling,alpha
0,"[l1, l2]","[1, 2, 3, 4]","[1, 2, 3, 4]",{'alpha': 0.01},0.209681,0.209681,0.209681,0.01
1,"[l1, l2]","[1, 2, 3, 4]","[1, 2, 3, 4]",{'alpha': 0.1},0.209693,0.209693,0.209693,0.10
2,"[l1, l2]","[1, 2, 3, 4]","[1, 2, 3, 4]",{'alpha': 1},0.209779,0.209779,0.209779,1.00
3,"[l1, l2]","[1, 2]","[1, 2]",{'alpha': 1},0.210773,0.210773,0.210773,1.00
4,"[l1, l2]","[1, 2]","[1, 2]",{'alpha': 0.1},0.210787,0.210787,0.210787,0.10
5,"[l1, l2]","[1, 2]","[1, 2]",{'alpha': 0.01},0.210788,0.210788,0.210788,0.01


In [180]:
import warnings

In [184]:
def test_output_bayesian_search_forecaster_multiseries_series_and_exog_dict_multiple_metrics_aggregated_with_mocked():
    """
    Test output of bayesian_search_forecaster_multiseries in ForecasterAutoregMultiSeries
    and ForecasterAutoregMultiSeriesCustom when series and exog are dictionaries and 
    multiple aggregated metrics (mocked done in Skforecast v0.12.0).
    """

    forecaster = ForecasterAutoregMultiSeries(
        regressor=LGBMRegressor(
            n_estimators=2, random_state=123, verbose=-1, max_depth=2
        ),
        lags=14,
        encoding="ordinal",
        dropna_from_series=False,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
    )

    lags_grid = [[5], [1, 7, 14]]

    def search_space(trial):
        search_space = {
            "n_estimators": trial.suggest_int("n_estimators", 2, 5),
            "max_depth": trial.suggest_int("max_depth", 2, 5),
            "lags": trial.suggest_categorical("lags", lags_grid),
        }

        return search_space

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=UserWarning, module="optuna")

        results_search, _ = bayesian_search_forecaster_multiseries(
            forecaster=forecaster,
            series=series_dict,
            exog=exog_dict,
            search_space=search_space,
            metric=['mean_absolute_error', 'mean_absolute_scaled_error'],
            aggregate_metric=['weighted_average', 'average', 'pooling'],
            initial_train_size=len(series_dict_train["id_1000"]),
            steps=24,
            refit=False,
            n_trials=3,
            return_best=False,
            show_progress=False,
            verbose=False,
            suppress_warnings=True,
        )

    expected = pd.DataFrame({
                    "levels": {
                        0: ["id_1000", "id_1001", "id_1002", "id_1003", "id_1004"],
                        1: ["id_1000", "id_1001", "id_1002", "id_1003", "id_1004"],
                        2: ["id_1000", "id_1001", "id_1002", "id_1003", "id_1004"],
                    },
                    "lags": {0: np.array([1, 7, 14]), 1: np.array([1, 7, 14]), 2: np.array([5])},
                    "params": {
                        0: {"n_estimators": 4, "max_depth": 3},
                        1: {"n_estimators": 3, "max_depth": 3},
                        2: {"n_estimators": 4, "max_depth": 3},
                    },
                    "mean_absolute_error__weighted_average": {
                        0: 749.8761502029433,
                        1: 760.659082077477,
                        2: 777.6874712018467,
                    },
                    "mean_absolute_error__average": {
                        0: 709.8836514262415,
                        1: 721.1848222120482,
                        2: 754.3537196425694,
                    },
                    "mean_absolute_error__pooling": {
                        0: 709.8836514262414,
                        1: 721.1848222120483,
                        2: 754.3537196425694,
                    },
                    "mean_absolute_scaled_error__weighted_average": {
                        0: 1.7214020008755428,
                        1: 1.7480018562024022,
                        2: 1.8159623522099073,
                    },
                    "mean_absolute_scaled_error__average": {
                        0: 2.0671251354627653,
                        1: 2.099920474995049,
                        2: 2.1945202856306465,
                    },
                    "mean_absolute_scaled_error__pooling": {
                        0: 1.6864677542505797,
                        1: 1.7133159005309597,
                        2: 1.7921151176255248,
                    },
                    "n_estimators": {0: 4, 1: 3, 2: 4},
                    "max_depth": {0: 3, 1: 3, 2: 3},
                })

    pd.testing.assert_frame_equal(expected, results_search)


test_output_bayesian_search_forecaster_multiseries_series_and_exog_dict_multiple_metrics_aggregated_with_mocked()

In [185]:
forecaster = ForecasterAutoregMultiSeries(
    regressor=LGBMRegressor(
        n_estimators=2, random_state=123, verbose=-1, max_depth=2
    ),
    lags=14,
    encoding="ordinal",
    dropna_from_series=False,
    transformer_series=StandardScaler(),
    transformer_exog=StandardScaler(),
)

lags_grid = [[5], [1, 7, 14]]

def search_space(trial):
    search_space = {
        "n_estimators": trial.suggest_int("n_estimators", 2, 5),
        "max_depth": trial.suggest_int("max_depth", 2, 5),
        "lags": trial.suggest_categorical("lags", lags_grid),
    }

    return search_space

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning, module="optuna")

    results_search, best_trial = bayesian_search_forecaster_multiseries(
        forecaster=forecaster,
        series=series_dict,
        exog=exog_dict,
        search_space=search_space,
        metric=['mean_absolute_error', 'mean_absolute_scaled_error'],
        aggregate_metric=['weighted_average', 'average', 'pooling'],
        initial_train_size=len(series_dict_train["id_1000"]),
        steps=24,
        refit=False,
        n_trials=3,
        return_best=False,
        show_progress=False,
        verbose=False,
        suppress_warnings=True,
    )

results_search

,levels,lags,params,mean_absolute_error__weighted_average,mean_absolute_error__average,mean_absolute_error__pooling,mean_absolute_scaled_error__weighted_average,mean_absolute_scaled_error__average,mean_absolute_scaled_error__pooling,n_estimators,max_depth
0,"[id_1000, id_1001, id_1002, id_1003, id_1004]","[1, 7, 14]","{'n_estimators': 4, 'max_depth': 3}",749.876150,709.883651,709.883651,1.721402,2.067125,1.686468,4,3
1,"[id_1000, id_1001, id_1002, id_1003, id_1004]","[1, 7, 14]","{'n_estimators': 3, 'max_depth': 3}",760.659082,721.184822,721.184822,1.748002,2.099920,1.713316,3,3
2,"[id_1000, id_1001, id_1002, id_1003, id_1004]",[5],"{'n_estimators': 4, 'max_depth': 3}",777.687471,754.353720,754.353720,1.815962,2.194520,1.792115,4,3


In [192]:
def test_results_output_bayesian_search_optuna_multiseries_ForecasterAutoregMultiSeries_with_multiple_metrics_aggregated():
    """
    Test output of _bayesian_search_optuna_multiseries in ForecasterAutoregMultiSeries
    with multiple metrics and aggregated metrics (mocked done in skforecast v0.12.0).
    """
    forecaster = ForecasterAutoregMultiSeries(
                     regressor = Ridge(random_state=123),
                     lags      = 2,
                     encoding  = 'onehot',
                     transformer_series = StandardScaler()
                 )

    steps = 3
    n_validation = 12

    def search_space(trial):
        search_space  = {
            'alpha': trial.suggest_float('alpha', 1e-2, 1.0),
            'lags' : trial.suggest_categorical('lags', [2, 4])
        }
        
        return search_space

    results = _bayesian_search_optuna_multiseries(
                  forecaster         = forecaster,
                  series             = series,
                  steps              = steps,
                  search_space       = search_space,
                  metric             = ['mean_absolute_error', 'mean_absolute_scaled_error'],
                  aggregate_metric   = ['weighted_average', 'average', 'pooling'],
                  refit              = True,
                  initial_train_size = len(series) - n_validation,
                  fixed_train_size   = True,
                  n_trials           = 10,
                  random_state       = 123,
                  return_best        = False,
                  verbose            = False
              )[0]
    
    expected_results = pd.DataFrame({
        "levels": {
            0: ["l1", "l2"],
            1: ["l1", "l2"],
            2: ["l1", "l2"],
            3: ["l1", "l2"],
            4: ["l1", "l2"],
            5: ["l1", "l2"],
            6: ["l1", "l2"],
            7: ["l1", "l2"],
            8: ["l1", "l2"],
            9: ["l1", "l2"],
        },
        "lags": {
            0: np.array([1, 2]),
            1: np.array([1, 2]),
            2: np.array([1, 2]),
            3: np.array([1, 2]),
            4: np.array([1, 2]),
            5: np.array([1, 2, 3, 4]),
            6: np.array([1, 2, 3, 4]),
            7: np.array([1, 2, 3, 4]),
            8: np.array([1, 2, 3, 4]),
            9: np.array([1, 2, 3, 4]),
        },
        "params": {
            0: {"alpha": 0.5558016213920624},
            1: {"alpha": 0.6995044937418831},
            2: {"alpha": 0.7406154516747153},
            3: {"alpha": 0.8509374761370117},
            4: {"alpha": 0.9809565564007693},
            5: {"alpha": 0.23598059857016607},
            6: {"alpha": 0.398196343012209},
            7: {"alpha": 0.4441865222328282},
            8: {"alpha": 0.53623586010342},
            9: {"alpha": 0.7252189487445193},
        },
        "mean_absolute_error__weighted_average": {
            0: 0.21324663796176382,
            1: 0.2132571094660072,
            2: 0.21326009091608622,
            3: 0.21326806055662118,
            4: 0.2132773952926551,
            5: 0.21476196207156512,
            6: 0.21477679099211167,
            7: 0.21478095843883202,
            8: 0.2147892513261171,
            9: 0.21480607764821474,
        },
        "mean_absolute_error__average": {
            0: 0.21324663796176382,
            1: 0.21325710946600718,
            2: 0.21326009091608622,
            3: 0.21326806055662115,
            4: 0.2132773952926551,
            5: 0.21476196207156514,
            6: 0.21477679099211167,
            7: 0.21478095843883202,
            8: 0.21478925132611706,
            9: 0.21480607764821472,
        },
        "mean_absolute_error__pooling": {
            0: 0.21324663796176382,
            1: 0.21325710946600726,
            2: 0.21326009091608622,
            3: 0.21326806055662115,
            4: 0.21327739529265513,
            5: 0.21476196207156514,
            6: 0.21477679099211167,
            7: 0.21478095843883202,
            8: 0.21478925132611706,
            9: 0.21480607764821472,
        },
        "mean_absolute_scaled_error__weighted_average": {
            0: 0.7850408674109868,
            1: 0.7850769204162228,
            2: 0.7850871846200614,
            3: 0.7851146198219785,
            4: 0.7851467509994321,
            5: 0.7897394420953061,
            6: 0.7897964331491425,
            7: 0.7898124493861658,
            8: 0.7898443200957621,
            9: 0.7899089846155858,
        },
        "mean_absolute_scaled_error__average": {
            0: 0.7850408674109867,
            1: 0.7850769204162228,
            2: 0.7850871846200613,
            3: 0.7851146198219786,
            4: 0.785146750999432,
            5: 0.7897394420953061,
            6: 0.7897964331491426,
            7: 0.7898124493861657,
            8: 0.7898443200957622,
            9: 0.7899089846155857,
        },
        "mean_absolute_scaled_error__pooling": {
            0: 0.7724806513327171,
            1: 0.7725185840968428,
            2: 0.7725293843257277,
            3: 0.7725582541506879,
            4: 0.7725920690001996,
            5: 0.7779698752966109,
            6: 0.7780235926931046,
            7: 0.7780386891653759,
            8: 0.7780687299436625,
            9: 0.7781296828776818,
        },
        "alpha": {
            0: 0.5558016213920624,
            1: 0.6995044937418831,
            2: 0.7406154516747153,
            3: 0.8509374761370117,
            4: 0.9809565564007693,
            5: 0.23598059857016607,
            6: 0.398196343012209,
            7: 0.4441865222328282,
            8: 0.53623586010342,
            9: 0.7252189487445193,
        },
    })

    pd.testing.assert_frame_equal(results, expected_results, check_dtype=False)


test_results_output_bayesian_search_optuna_multiseries_ForecasterAutoregMultiSeries_with_multiple_metrics_aggregated()

  0%|          | 0/10 [00:00<?, ?it/s]

In [193]:
forecaster = ForecasterAutoregMultiSeries(
                    regressor = Ridge(random_state=123),
                    lags      = 2,
                    encoding  = 'onehot',
                    transformer_series = StandardScaler()
                )

steps = 3
n_validation = 12

def search_space(trial):
    search_space  = {
        'alpha': trial.suggest_float('alpha', 1e-2, 1.0),
        'lags' : trial.suggest_categorical('lags', [2, 4])
    }
    
    return search_space

results = _bayesian_search_optuna_multiseries(
                forecaster         = forecaster,
                series             = series,
                steps              = steps,
                search_space       = search_space,
                metric             = ['mean_absolute_error', 'mean_absolute_scaled_error'],
                aggregate_metric   = ['weighted_average', 'average', 'pooling'],
                refit              = True,
                initial_train_size = len(series) - n_validation,
                fixed_train_size   = True,
                n_trials           = 10,
                random_state       = 123,
                return_best        = False,
                verbose            = False,
                show_progress      = False
            )[0]

results

  0%|          | 0/10 [00:00<?, ?it/s]

,levels,lags,params,mean_absolute_error__weighted_average,mean_absolute_error__average,mean_absolute_error__pooling,mean_absolute_scaled_error__weighted_average,mean_absolute_scaled_error__average,mean_absolute_scaled_error__pooling,alpha
0,"[l1, l2]","[1, 2]",{'alpha': 0.5558016213920624},0.213247,0.213247,0.213247,0.785041,0.785041,0.772481,0.555802
1,"[l1, l2]","[1, 2]",{'alpha': 0.6995044937418831},0.213257,0.213257,0.213257,0.785077,0.785077,0.772519,0.699504
2,"[l1, l2]","[1, 2]",{'alpha': 0.7406154516747153},0.213260,0.213260,0.213260,0.785087,0.785087,0.772529,0.740615
3,"[l1, l2]","[1, 2]",{'alpha': 0.8509374761370117},0.213268,0.213268,0.213268,0.785115,0.785115,0.772558,0.850937
4,"[l1, l2]","[1, 2]",{'alpha': 0.9809565564007693},0.213277,0.213277,0.213277,0.785147,0.785147,0.772592,0.980957
5,"[l1, l2]","[1, 2, 3, 4]",{'alpha': 0.23598059857016607},0.214762,0.214762,0.214762,0.789739,0.789739,0.777970,0.235981
6,"[l1, l2]","[1, 2, 3, 4]",{'alpha': 0.398196343012209},0.214777,0.214777,0.214777,0.789796,0.789796,0.778024,0.398196
7,"[l1, l2]","[1, 2, 3, 4]",{'alpha': 0.4441865222328282},0.214781,0.214781,0.214781,0.789812,0.789812,0.778039,0.444187
8,"[l1, l2]","[1, 2, 3, 4]",{'alpha': 0.53623586010342},0.214789,0.214789,0.214789,0.789844,0.789844,0.778069,0.536236
9,"[l1, l2]","[1, 2, 3, 4]",{'alpha': 0.7252189487445193},0.214806,0.214806,0.214806,0.789909,0.789909,0.778130,0.725219


In [ ]:
{
    "levels": {
        0: ["l1", "l2"],
        1: ["l1", "l2"],
        2: ["l1", "l2"],
        3: ["l1", "l2"],
        4: ["l1", "l2"],
        5: ["l1", "l2"],
        6: ["l1", "l2"],
        7: ["l1", "l2"],
        8: ["l1", "l2"],
        9: ["l1", "l2"],
    },
    "lags": {
        0: np.array([1, 2]),
        1: np.array([1, 2]),
        2: np.array([1, 2]),
        3: np.array([1, 2]),
        4: np.array([1, 2]),
        5: np.array([1, 2, 3, 4]),
        6: np.array([1, 2, 3, 4]),
        7: np.array([1, 2, 3, 4]),
        8: np.array([1, 2, 3, 4]),
        9: np.array([1, 2, 3, 4]),
    },
    "params": {
        0: {"alpha": 0.5558016213920624},
        1: {"alpha": 0.6995044937418831},
        2: {"alpha": 0.7406154516747153},
        3: {"alpha": 0.8509374761370117},
        4: {"alpha": 0.9809565564007693},
        5: {"alpha": 0.23598059857016607},
        6: {"alpha": 0.398196343012209},
        7: {"alpha": 0.4441865222328282},
        8: {"alpha": 0.53623586010342},
        9: {"alpha": 0.7252189487445193},
    },
    "mean_absolute_error__weighted_average": {
        0: 0.21324663796176382,
        1: 0.2132571094660072,
        2: 0.21326009091608622,
        3: 0.21326806055662118,
        4: 0.2132773952926551,
        5: 0.21476196207156512,
        6: 0.21477679099211167,
        7: 0.21478095843883202,
        8: 0.2147892513261171,
        9: 0.21480607764821474,
    },
    "mean_absolute_error__average": {
        0: 0.21324663796176382,
        1: 0.21325710946600718,
        2: 0.21326009091608622,
        3: 0.21326806055662115,
        4: 0.2132773952926551,
        5: 0.21476196207156514,
        6: 0.21477679099211167,
        7: 0.21478095843883202,
        8: 0.21478925132611706,
        9: 0.21480607764821472,
    },
    "mean_absolute_error__pooling": {
        0: 0.21324663796176382,
        1: 0.21325710946600726,
        2: 0.21326009091608622,
        3: 0.21326806055662115,
        4: 0.21327739529265513,
        5: 0.21476196207156514,
        6: 0.21477679099211167,
        7: 0.21478095843883202,
        8: 0.21478925132611706,
        9: 0.21480607764821472,
    },
    "mean_absolute_scaled_error__weighted_average": {
        0: 0.7850408674109868,
        1: 0.7850769204162228,
        2: 0.7850871846200614,
        3: 0.7851146198219785,
        4: 0.7851467509994321,
        5: 0.7897394420953061,
        6: 0.7897964331491425,
        7: 0.7898124493861658,
        8: 0.7898443200957621,
        9: 0.7899089846155858,
    },
    "mean_absolute_scaled_error__average": {
        0: 0.7850408674109867,
        1: 0.7850769204162228,
        2: 0.7850871846200613,
        3: 0.7851146198219786,
        4: 0.785146750999432,
        5: 0.7897394420953061,
        6: 0.7897964331491426,
        7: 0.7898124493861657,
        8: 0.7898443200957622,
        9: 0.7899089846155857,
    },
    "mean_absolute_scaled_error__pooling": {
        0: 0.7724806513327171,
        1: 0.7725185840968428,
        2: 0.7725293843257277,
        3: 0.7725582541506879,
        4: 0.7725920690001996,
        5: 0.7779698752966109,
        6: 0.7780235926931046,
        7: 0.7780386891653759,
        8: 0.7780687299436625,
        9: 0.7781296828776818,
    },
    "alpha": {
        0: 0.5558016213920624,
        1: 0.6995044937418831,
        2: 0.7406154516747153,
        3: 0.8509374761370117,
        4: 0.9809565564007693,
        5: 0.23598059857016607,
        6: 0.398196343012209,
        7: 0.4441865222328282,
        8: 0.53623586010342,
        9: 0.7252189487445193,
    },
}